In [1]:
import numpy as np, pandas as pd

!date
!whoami

Mon Mar 20 12:26:24 PDT 2023
zmbc


# Find data

In [2]:
output_dir = '/ihme/scratch/users/zmbc/vivarium_results/united_states_of_america/2023_03_20_11_09_48/final_results/2023_03_20_12_04_48'

!ls -lh $output_dir

total 32K
drwxrwxr-x 2 zmbc Domain Users 512 Mar 20 12:08 decennial_census_observer
drwxrwxr-x 2 zmbc Domain Users 512 Mar 20 12:08 household_survey_observer_acs
drwxrwxr-x 2 zmbc Domain Users 512 Mar 20 12:08 household_survey_observer_cps
drwxrwxr-x 2 zmbc Domain Users 512 Mar 20 12:09 social_security_observer
drwxrwxr-x 2 zmbc Domain Users 512 Mar 20 12:09 tax_1040_observer
drwxrwxr-x 2 zmbc Domain Users 512 Mar 20 12:09 tax_dependents_observer
drwxrwxr-x 2 zmbc Domain Users 512 Mar 20 12:09 tax_w2_observer
drwxrwxr-x 2 zmbc Domain Users 512 Mar 20 12:09 wic_observer


In [3]:
!ls -lh $output_dir/decennial_census_observer

total 508K
-rwxr-xr-x 1 zmbc Domain Users 502K Mar 20 12:08 decennial_census_observer.csv.bz2


# Load decennial Census

In [4]:
%%time

df_census = pd.read_csv(f'{output_dir}/decennial_census_observer/decennial_census_observer.csv.bz2')
df_census

CPU times: user 167 ms, sys: 9.58 ms, total: 177 ms
Wall time: 211 ms


,first_name,last_name,age,unit_number,guardian_1,sex,zipcode,state,year,housing_type,street_name,simulant_id,race_ethnicity,middle_initial,relation_to_household_head,guardian_2,street_number,city,date_of_birth
0,Maureen,Shipp,76.024932,NaN,0_-1,Female,90210,USA,2020,Standard,commonwealth avnue,0_4,Black,M,Reference person,0_-1,1312,Anytown,1944-03-23
1,Richard,Shipp,69.075260,NaN,0_-1,Male,90210,USA,2020,Standard,commonwealth avnue,0_5,Black,N,Opp-sex spouse,0_-1,1312,Anytown,1951-03-05
2,Darlene,Holt,66.661179,NaN,0_-1,Female,90210,USA,2020,Standard,bancroft st,0_2464,Black,C,Reference person,0_-1,46,Anytown,1953-08-03
3,Tabatha,Holt,42.860033,NaN,0_-1,Female,90210,USA,2020,Standard,janis ne,0_3558,Black,S,Reference person,0_-1,9112,Anytown,1977-05-22
4,Alice,Holt,25.293532,NaN,0_-1,Female,90210,USA,2020,Standard,janis ne,0_3559,Black,M,Biological child,0_-1,9112,Anytown,1994-12-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19949,John,Wasser,66.493182,NaN,0_-1,Male,90210,USA,2030,Standard,n cr 400 e,0_17995,White,S,Reference person,0_-1,5700,Anytown,1963-09-21
19950,Clyde,Hughes,77.426139,NaN,0_-1,Male,90210,USA,2030,Standard,mammoth springs dr,0_18073,Black,K,Reference person,0_-1,21,Anytown,1952-10-15
19951,Stephanie,Walker,2.432617,NaN,0_14366,Female,90210,USA,2030,Standard,high pine meadows r,0_22588,Black,C,Reference person,0_-1,8728,Anytown,2027-11-11
19952,Austin,Foster,23.517320,NaN,0_-1,Male,90210,USA,2030,Standard,ranch loop,0_22759,White,D,Reference person,0_-1,1900,Anytown,2006-10-10


In [5]:
df_census.groupby('year').size()

year
2020    9979
2030    9975
dtype: int64

## Overlap between consecutive Censuses

This is probably too little overlap, but close enough? Definitely enough for a meaningful PRL exercise.

In [6]:
df_census[df_census.year == 2030].assign(in_2020_census=lambda df: df.simulant_id.isin(df_census[df_census.year == 2020].simulant_id)).in_2020_census.value_counts(dropna=False)

True     5703
False    4272
Name: in_2020_census, dtype: int64

# 1040

In [7]:
%%time

df_1040 = pd.read_csv(f'{output_dir}/tax_1040_observer/tax_1040_observer.csv.bz2')
df_1040

CPU times: user 563 ms, sys: 31.5 ms, total: 595 ms
Wall time: 618 ms


,first_name,last_name,mailing_address_street_name,age,tax_year,joint_filer,housing_type,itin,simulant_id,mailing_address_zipcode,ssn,middle_initial,mailing_address_street_number,mailing_address_state,mailing_address_unit_number,mailing_address_city,mailing_address_po_box,date_of_birth
0,Maureen,Shipp,commonwealth avnue,76.024932,2019,False,Standard,NaN,0_4,90210,828-99-4653,M,1312,USA,NaN,Anytown,0,1944-03-23
1,Richard,Shipp,commonwealth avnue,69.075260,2019,True,Standard,NaN,0_5,90210,885-38-0858,N,1312,USA,NaN,Anytown,0,1951-03-05
2,Darlene,Holt,bancroft st,66.661179,2019,False,Standard,NaN,0_2464,90210,584-19-7087,C,46,USA,NaN,Anytown,0,1953-08-03
3,Tabatha,Holt,janis ne,42.860033,2019,False,Standard,NaN,0_3558,90210,813-32-2963,S,9112,USA,NaN,Anytown,0,1977-05-22
4,Alice,Holt,janis ne,25.293532,2019,False,Standard,NaN,0_3559,90210,607-79-1610,M,9112,USA,NaN,Anytown,0,1994-12-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116312,Michael,Rubio,ocean avenue,13.883349,2029,False,Standard,NaN,0_12669,90210,088-27-8587,B,2569,USA,NaN,Anytown,0,2016-05-01
116313,Alana,Rubio,ocean avenue,27.497492,2029,False,Standard,NaN,0_12670,90210,028-47-6626,K,2569,USA,NaN,Anytown,0,2002-09-20
116314,Emily,Rubio,ocean avenue,26.185562,2029,False,Standard,NaN,0_12671,90210,796-80-2013,C,2569,USA,NaN,Anytown,0,2004-01-12
116315,Joaquin,Rubio,ocean avenue,9.634704,2029,False,Standard,NaN,0_20143,90210,482-54-0254,B,2569,USA,NaN,Anytown,0,2020-08-28


## Overlap between consecutive tax years

In [8]:
df_1040.tax_year.value_counts(dropna=False)

2029    11160
2028    11063
2027    10974
2026    10860
2025    10735
2024    10525
2023    10381
2022    10337
2021    10213
2020    10083
2019     9986
Name: tax_year, dtype: int64

In [9]:
df_1040[df_1040.tax_year == 2028].assign(in_2027_taxes=lambda df: df.simulant_id.isin(df_1040[df_1040.tax_year == 2027].simulant_id)).in_2027_taxes.value_counts(dropna=False)

True     10429
False      634
Name: in_2027_taxes, dtype: int64

## Why does taxes have more rows than Census?

I think because it is including people who have died, and does not implement the minimum income for filing (!).

In [10]:
df_1040[df_1040.tax_year == 2029].simulant_id.is_unique

True

In [11]:
not_in_census = df_1040[(df_1040.tax_year == 2029) & (~df_1040.simulant_id.isin(df_census[df_census.year == 2030].simulant_id))]
(pd.to_datetime(not_in_census.date_of_birth).dt.year + not_in_census.age).describe()

count    1185.000000
mean     2024.876258
std         2.881087
min      2019.255773
25%      2022.376241
50%      2025.003906
75%      2027.346760
max      2030.004095
dtype: float64